In [ ]:
# @title 1. Initialize Workspace & Hardware Profile
import os
import torch
import sys
from google.colab import drive

# --- CONFIGURATION ---
REPO_URL = "https://github.com/BorisKlimchenko/Adaptive-Motion-Lab.git"
REPO_DIR = "/content/Adaptive-Motion-Lab"

# 1. Secure Drive Mount
if not os.path.exists('/content/drive'):
    print("📂 Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("✅ Drive already mounted.")

# 2. Repository Synchronization
if not os.path.exists(REPO_DIR):
    print(f"⬇️ Cloning Repository: {REPO_URL}...")
    !git clone {REPO_URL}
else:
    print("🔄 Repository exists. Pulling latest changes...")
    !cd {REPO_DIR} && git pull

# 3. Hardware Profiling
print("\n--- Hardware Acceleration Profile ---")
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Detected: {device_name} ({vram:.2f} GB VRAM)")
    
    if "A100" in device_name:
        print("🚀 Status: DATA CENTER MODE (High Performance)")
    elif "T4" in device_name:
        print("⚠️ Status: EFFICIENCY MODE (Restricted VRAM)")
else:
    print("❌ CRITICAL: No GPU detected. Change Runtime Type to GPU.")

# Set working directory context
%cd {REPO_DIR}

In [ ]:
# @title 2. Smart Environment Orchestrator (Auto-Repair)
import subprocess
import sys
import os
import pkg_resources

# --- CONFIGURATION (Single Source of Truth) ---
# Defines the Golden Standard for reproducibility.
TARGET_ENV = {
    "torch": "2.5.1+cu121",
    "torchvision": "0.20.1+cu121",
    "torchaudio": "2.5.1+cu121",
    "xformers": "0.0.28.post3"
}

# PyTorch Index URL for CUDA 12.1
INDEX_URL = "https://download.pytorch.org/whl/cu121"

def run_command(command, message):
    """Executes shell command with status logging."""
    print(f"🔄 {message}...")
    try:
        subprocess.check_call(command, shell=True)
        print(f"✅ Success: {message}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error: {message} failed.")
        raise e

def get_installed_version(package_name):
    """Safely retrieves the installed version."""
    try:
        return pkg_resources.get_distribution(package_name).version
    except pkg_resources.DistributionNotFound:
        return None

def setup_environment():
    print("🔍 Analyzing Runtime Environment...")
    
    needs_install = False
    current_torch = get_installed_version("torch")
    target_base = TARGET_ENV["torch"].split('+')[0]
    
    # 1. Audit Phase
    if current_torch is None:
        print("⚠️ Torch not found. Initialization required.")
        needs_install = True
    elif not current_torch.startswith(target_base):
        print(f"⚠️ Version Mismatch! Target: {target_base}..., Found: {current_torch}")
        needs_install = True
    else:
        print(f"✅ Core Environment (Torch {current_torch}) is optimized.")

    # 2. Execution Phase
    if needs_install:
        print("🚑 Initiating Environment Alignment...")
        
        # A. Clean Slate Protocol
        print("   -> Removing conflicting libraries...")
        subprocess.run("pip uninstall -y torch torchvision torchaudio xformers", shell=True, check=False)
        
        # B. Install Golden Standard
        install_cmd = (
            f"pip install "
            f"torch=={TARGET_ENV['torch']} "
            f"torchvision=={TARGET_ENV['torchvision']} "
            f"torchaudio=={TARGET_ENV['torchaudio']} "
            f"xformers=={TARGET_ENV['xformers']} "
            f"--index-url {INDEX_URL}"
        )
        run_command(install_cmd, "Installing Core Stack (Torch + xFormers)")
        
    # 3. Project Dependencies
    run_command("pip install -r requirements.txt", "Verifying Project Requirements")

    # 4. Path Injection (Fixes 'ModuleNotFoundError' after restart)
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())
        print(f"📂 System Path Updated: {os.getcwd()}")

if __name__ == "__main__":
    setup_environment()

In [ ]:
# @title 3. Launch Inference Pipeline
import os
import sys

# --- Runtime Path Fix ---
# Ensures Python sees the project modules even if Colab context shifted.
PROJECT_ROOT = "/content/Adaptive-Motion-Lab"
if os.path.exists(PROJECT_ROOT):
    os.chdir(PROJECT_ROOT)
    if PROJECT_ROOT not in sys.path:
        sys.path.append(PROJECT_ROOT)

print(f"🚀 Initializing Inference Engine in: {os.getcwd()}")

# --- Execute Main Pipeline ---
# Use !python to run as a subprocess, ensuring memory cleanup after run.
!python main.py --prompts configs/default_scene.json